In [1]:
import os
import numpy as np
import tensorflow as tf

In [2]:
data_path = "C:\\Users\\Owner6\\workspace\\midterm\\after_data"

train_path = os.path.join(data_path, 'train')
validation_path = os.path.join(data_path, 'validation')
test_path = os.path.join(data_path, 'test')

train_len = len(os.listdir( os.path.join( train_path, 'normal' ) )) +\
    len(os.listdir( os.path.join( train_path, 'bacteria' ) )) +\
    len(os.listdir( os.path.join( train_path, 'virus' ) )) +\
    len(os.listdir( os.path.join( train_path, 'covid' ) ))
    
validation_len = len(os.listdir( os.path.join( validation_path, 'normal' ) )) +\
    len(os.listdir( os.path.join( validation_path, 'bacteria' ) )) +\
    len(os.listdir( os.path.join( validation_path, 'virus' ) )) +\
    len(os.listdir( os.path.join( validation_path, 'covid' ) ))
    
test_len = len(os.listdir( os.path.join( test_path, 'normal' ) )) +\
    len(os.listdir( os.path.join( test_path, 'bacteria' ) )) +\
    len(os.listdir( os.path.join( test_path, 'virus' ) )) +\
    len(os.listdir( os.path.join( test_path, 'covid' ) ))

print(f'train : {train_len}, validation : {validation_len}, test : {test_len}')

train : 11507, validation : 1645, test : 3286


In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
data_gen = ImageDataGenerator(rescale=1./255)

train_data = data_gen.flow_from_directory(
    train_path, class_mode='categorical', color_mode='rgb',
    target_size=(1024,1024), batch_size=11507 # 11507 = 37 * 311
)

validation_data = data_gen.flow_from_directory(
    validation_path, class_mode='categorical', color_mode='rgb',
    target_size=(1024,1024), batch_size=1645 # 1645 = 35 * 47
)

test_data = data_gen.flow_from_directory(
    test_path, class_mode='categorical', color_mode='rgb',
    target_size=(1024,1024), batch_size=3286 # 3286 = 31 * 106
)

train_x, train_y = train_data.next()
validation_x, validation_y = validation_data.next()
# test_x, test_y = test_data.next()

train_x.shape, train_y.shape

Found 11507 images belonging to 4 classes.
Found 1645 images belonging to 4 classes.
Found 3286 images belonging to 4 classes.


In [ ]:
from tensorflow.keras.applications import VGG16

conv_base = VGG16(weights='imagenet',
                  include_top=False,
                  input_shape=(1024, 1024, 3))

In [ ]:
conv_base.trainable = False
conv_base.summary()

Error: Kernel is dead

In [ ]:
from tensorflow.keras import models, layers

model = models.Sequential()
model.add(conv_base)
model.add(layers.Conv2D(1024, (3,3), activation='relu', padding='same'))
model.add(layers.Conv2D(1024, (3,3), activation='relu', padding='same'))
model.add(layers.Conv2D(1024, (3,3), activation='relu', padding='same'))
model.add(layers.Conv2D(1024, (3,3), activation='relu', padding='same'))
model.add(layers.MaxPool2D((2,2)))
model.add(layers.Conv2D(1024, (3,3), activation='relu', padding='same'))
model.add(layers.Conv2D(1024, (3,3), activation='relu', padding='same'))
model.add(layers.Conv2D(1024, (3,3), activation='relu', padding='same'))
model.add(layers.Conv2D(1024, (3,3), activation='relu', padding='same'))
model.add(layers.MaxPool2D((2,2)))
model.add(layers.Flatten())
model.add(layers.Dense(1024, activation='relu'))
model.add(layers.Dropout(0.3))
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dropout(0.3))
model.add(layers.Dense(4, activation='softmax'))

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Functional)           (None, 32, 32, 512)       14714688  
_________________________________________________________________
conv2d (Conv2D)              (None, 32, 32, 1024)      4719616   
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 1024)      9438208   
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 1024)      9438208   
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 32, 32, 1024)      9438208   
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 16, 16, 1024)      0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 16, 16, 1024)      9

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint

checkpoint_filepath = 'model/checkpoint'
model_checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_filepath,
    monitor='val_accuracy',
    verbose=1,
    save_best_only=True,
    mode='auto'
)

In [ ]:
model.compile(optimizer="rmsprop", \
    loss='categorical_crossentropy', 
    metrics=['accuracy'])

history = model.fit(
    train_x, train_y, epochs=30, batch_size=100, 
    validation_data=(validation_x, validation_y),
    callbacks=[model_checkpoint_callback]
)

C:\Users\Owner6\Anaconda3\envs\jh\lib\site-packages\tensorflow\python\keras\engine\training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/30


In [ ]:
# history = model.fit_generator(train_data, steps_per_epoch=311, epochs=30,\
#     validation_data=validation_data, validation_steps=47, callbacks=[model_checkpoint_callback])